In [3]:
import pandas as pd
from datetime import datetime, timedelta
import getpass

# -----------------------------
# STEP 1: CREATE SAMPLE INPUT DATA
# -----------------------------

# MASTER INPUT
master_df = pd.DataFrame({
    'Version': ['V1', 'V1'],
    'Version Name': ['Ver1', 'Ver1'],
    'Location': ['Loc1', 'Loc1'],
    'EOL Group ID': ['EG1', 'EG1'],
    'Sub Location': ['SLoc1', 'SLoc1'],
    'Item': ['Item1', 'Item2'],
    'EOL Group Item Site ASN': [1, 0],
    'EOL Group Detail_Delete Flag': [False, False]
})

# DETAIL INPUT
detail_df = pd.DataFrame({
    'Version': ['V1', 'V1'],
    'Version Name': ['Ver1', 'Ver1'],
    'Location': ['Loc1', 'Loc1'],
    'EOL Group ID': ['EG1', 'EG1'],
    'Sub Location': ['SLoc1', 'SLoc1'],
    'Item': ['Item1', 'Item2'],
    'EOL Group Const EOP Min Date': [pd.Timestamp('2025-07-20'), pd.Timestamp('2025-07-10')],
    'EOL Group Const EOP Max Date': [pd.Timestamp('2025-07-30'), pd.Timestamp('2025-07-15')],
    'EOL Group Const Apply Flag': [True, True],
    'EOL Group Const Constraint Start Date': [pd.Timestamp('2025-07-01'), pd.Timestamp('2025-07-01')],
    'EOL Group Const Constraint End Date': [pd.Timestamp('2025-08-01'), pd.Timestamp('2025-08-01')],
    'EOL Group Const Constraint Qty': [100, 200],
    'EOL Group Const Demand Qty': [50, 150],
    'EOL Group Const Demand Short Qty': [10, 20],
    'EOL Group Const Prod Result Qty': [60, 160],
    'EOL Group Const Prod Plan Qty': [70, 170],
    'EOL Group Const Pegging Inv Qty': [30, 130],
    'EOL Group Const Creation time': [pd.Timestamp.now(), pd.Timestamp.now()],
    'EOL Group Const Created by': [getpass.getuser(), getpass.getuser()],
    'Master_EOP_Plan_Date': [pd.Timestamp('2025-07-25'), pd.Timestamp('2025-07-12')],
    'Master_EOP_Change_Date': [pd.Timestamp('2025-07-27'), pd.Timestamp('2025-07-14')],
})

# DEMAND INPUT
demand_df = pd.DataFrame({
    'Version': ['V1', 'V1'],
    'Location': ['Loc1', 'Loc1'],
    'Item': ['Item1', 'Item2'],
    'EOL Demand_Demand Qty': [100, 200],
    'EOL Demand_Demand Short Qty': [20, 30]
})

# PLAN INPUT
plan_df = pd.DataFrame({
    'Version': ['V1', 'V1'],
    'Location': ['Loc1', 'Loc1'],
    'Sub Location': ['SLoc1', 'SLoc1'],
    'Item': ['Item1', 'Item2'],
    'EOL Plan Result_Prod Result Qty': [100, 200],
    'EOL Plan Result_Prod Plan Qty': [100, 200],
    'EOL Plan Result_Pegging Inv Qty': [50, 150]
})

In [6]:
print(plan_df)

  Version Location Sub Location   Item  EOL Plan Result_Prod Result Qty  \
0      V1     Loc1        SLoc1  Item1                              100   
1      V1     Loc1        SLoc1  Item2                              200   

   EOL Plan Result_Prod Plan Qty  EOL Plan Result_Pegging Inv Qty  
0                            100                               50  
1                            200                              150  


In [3]:
# -----------------------------
# STEP 2: FILTER MASTER BY FLAGS
# -----------------------------
filtered_master = master_df[
    (master_df['EOL Group Item Site ASN'] == 1) &
    (~master_df['EOL Group Detail_Delete Flag'])
]

# -----------------------------
# STEP 3: MERGE DETAIL WITH DEMAND & PLAN
# -----------------------------
detail_with_demand = detail_df.merge(
    demand_df,
    on=['Version', 'Location', 'Item'],
    how='left'
)

detail_full = detail_with_demand.merge(
    plan_df,
    on=['Version', 'Location', 'Sub Location', 'Item'],
    how='left'
)


In [4]:
# -----------------------------
# STEP 4: APPLY EOL GROUP CONSTRAINT LOGIC
# -----------------------------
# Compute actual min/max of master EOP dates
actual_min_date = detail_full[['Master_EOP_Plan_Date', 'Master_EOP_Change_Date']].min(axis=1)
actual_max_date = detail_full[['Master_EOP_Plan_Date', 'Master_EOP_Change_Date']].max(axis=1)

# Constraint checks
mask_min_date = detail_full['EOL Group Const EOP Min Date'] <= actual_min_date
mask_max_date = detail_full['EOL Group Const EOP Max Date'] <= actual_max_date
mask_apply_flag = detail_full['EOL Group Const Apply Flag'].astype(bool)
mask_demand_qty = detail_full['EOL Group Const Demand Qty'] <= detail_full['EOL Demand_Demand Qty']
mask_short_qty = detail_full['EOL Group Const Demand Short Qty'] <= detail_full['EOL Demand_Demand Short Qty']
mask_prod_result = detail_full['EOL Group Const Prod Result Qty'] <= detail_full['EOL Plan Result_Prod Result Qty']
mask_prod_plan = detail_full['EOL Group Const Prod Plan Qty'] <= detail_full['EOL Plan Result_Prod Plan Qty']
mask_inv_qty = detail_full['EOL Group Const Pegging Inv Qty'] <= detail_full['EOL Plan Result_Pegging Inv Qty']
mask_time = detail_full['EOL Group Const Creation time'] <= (pd.Timestamp.now() + pd.Timedelta(hours=9))
mask_user = detail_full['EOL Group Const Created by'] == getpass.getuser()

# Combine all masks
final_mask = (
    mask_min_date & mask_max_date & mask_apply_flag &
    mask_demand_qty & mask_short_qty &
    mask_prod_result & mask_prod_plan & mask_inv_qty &
    mask_time & mask_user
)


In [5]:
# Apply the mask
filtered_detail = detail_full[final_mask]

# -----------------------------
# STEP 5: HANDLE EMPTY CASE (NO MATCHES)
# -----------------------------
if filtered_detail.empty:
    print("⚠️ No matching input. Applying to ALL detail records.")
    filtered_detail = detail_full.copy()
else:
    print("✅ Filtered detail rows found based on constraints.")

# -----------------------------
# STEP 6: OUTPUT
# -----------------------------
print("\n🔍 Final Processed Data:")
print(filtered_detail[['Version', 'Item', 'EOL Group Const Demand Qty', 'EOL Demand_Demand Qty']])

⚠️ No matching input. Applying to ALL detail records.

🔍 Final Processed Data:
  Version   Item  EOL Group Const Demand Qty  EOL Demand_Demand Qty
0      V1  Item1                          50                    100
1      V1  Item2                         150                    200


In [1]:
# Apply the mask
filtered_detail = detail_full[final_mask]

# -----------------------------
# STEP 5: HANDLE EMPTY CASE (NO MATCHES)
# -----------------------------
if filtered_detail.empty:
    print("⚠️ No matching input. Applying to ALL detail records.")
    filtered_detail = detail_full.copy()
else:
    print("✅ Filtered detail rows found based on constraints.")

# -----------------------------
# STEP 6: OUTPUT
# -----------------------------
print("\n🔍 Final Processed Data:")
print(filtered_detail[['Version', 'Item', 'EOL Group Const Demand Qty', 'EOL Demand_Demand Qty']])

NameError: name 'detail_full' is not defined